In [ ]:
import os
from tqdm import tqdm

INPUT_TXT = "combined.txt"
SHARD_DIR = "text_shards"
NUM_SHARDS = 100

os.makedirs(SHARD_DIR, exist_ok=True)

# Count lines (single pass)
print("Counting lines...")
with open(INPUT_TXT, "r", encoding="utf-8") as f:
    total_lines = sum(1 for _ in f)

lines_per_shard = total_lines // NUM_SHARDS
print(f"Total lines: {total_lines:,}")
print(f"Lines per shard: {lines_per_shard:,}")

writers = []
for i in range(NUM_SHARDS):
    f = open(f"{SHARD_DIR}/shard_{i}.txt", "w", encoding="utf-8")
    writers.append(f)

with open(INPUT_TXT, "r", encoding="utf-8") as f:
    for i, line in enumerate(tqdm(f, total=total_lines)):
        shard_id = min(i // lines_per_shard, NUM_SHARDS - 1)
        writers[shard_id].write(line)

for f in writers:
    f.close()

print("✓ Text sharding complete")